In [1]:
#Data Preprocess for benchmark
#Read in InterPro's json output, and convert to protein domain information
import json
import os
import numpy as np
import pandas as pd

In [2]:
json_path = "../dataset/ProteinGym/InterPro/"
filename = "substitution_part1.json"
#os.file.exist(json_path+filename)
with open(json_path+filename, 'r') as j:
     contents = json.loads(j.read())
reference_file = "../dataset/ProteinGym/reference_files/DMS_substitutions.csv"
df = pd.read_csv(reference_file)
df.head()

,DMS_id,DMS_filename,UniProt_ID,taxon,source_organism,target_seq,seq_len,includes_multiple_mutants,DMS_total_number_mutants,DMS_number_single_mutants,...,MSA_num_significant_L,raw_DMS_filename,raw_DMS_phenotype_name,raw_DMS_directionality,raw_DMS_mutant_column,weight_file_name,pdb_file,ProteinGym_version,raw_mut_offset,coarse_selection_type
0,A0A140D2T1_ZIKV_Sourisseau_2019,A0A140D2T1_ZIKV_Sourisseau_2019.csv,A0A140D2T1_ZIKV,Virus,Zika virus,MKNPKKKSGGFRIVNMLKRGVARVNPLGGLKRLPAGLLLGHGPIRM...,3423,False,9576,9576,...,0.661972,A0A140D2T1_ZIKV_Sourisseau_growth_2019.csv,effect,1,mutant,A0A140D2T1_ZIKV_theta_0.01.npy,A0A140D2T1_ZIKV.pdb,0.1,NaN,OrganismalFitness
1,A0A192B1T2_9HIV1_Haddox_2018,A0A192B1T2_9HIV1_Haddox_2018.csv,A0A192B1T2_9HIV1,Virus,HIV,MRVKGIQMNSQHLLRWGIMILGMIMICSVAGNLWVTVYYGVPVWKD...,852,False,12577,12577,...,2.835714,A0A192B1T2_9HIV1_Haddox_2018.csv,fitness,1,mutant,A0A192B1T2_9HIV1_theta_0.01.npy,A0A192B1T2_9HIV1.pdb,0.1,NaN,OrganismalFitness
2,A0A1I9GEU1_NEIME_Kennouche_2019,A0A1I9GEU1_NEIME_Kennouche_2019.csv,A0A1I9GEU1_NEIME,Prokaryote,Neisseria meningitidis,FTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSA...,161,False,922,922,...,0.521739,A0A1I9GEU1_NEIME_Kennouche_2019.csv,piliation_log2_ratio,1,mutants,A0A1I9GEU1_NEIME_theta_0.2.npy,A0A1I9GEU1_NEIME.pdb,0.1,NaN,Activity
3,A0A247D711_LISMN_Stadelmann_2021,A0A247D711_LISMN_Stadelmann_2021.csv,A0A247D711_LISMN,Eukaryote,Listeria monocytogenes,MNINDLIREIKNKDYTVKLSGTDSNSITQLIIRVNNDGNEYVISES...,87,False,1653,1653,...,2.402299,A0A247D711_LISMN_Stadelmann_2021.csv,mean_prediction,1,mutant,A0A247D711_LISMN_b03_theta_0.2.npy,A0A247D711_LISMN.pdb,1.0,NaN,Activity
4,A0A2Z5U3Z0_9INFA_Doud_2016,A0A2Z5U3Z0_9INFA_Doud_2016.csv,A0A2Z5U3Z0_9INFA,Virus,influenza H1N1,MKAKLLVLLYAFVATDADTICIGYHANNSTDTVDTILEKNVAVTHS...,565,False,10715,10715,...,1.691042,A0A2Z5U3Z0_9INFA_Doud_2016.csv,transformed_pref,1,mutant,A0A2Z5U3Z0_9INFA_theta_0.01.npy,A0A2Z5U3Z0_9INFA.pdb,0.1,NaN,OrganismalFitness


In [3]:
id = contents["results"][i]['xref'][0]['name']
df[df["DMS_id"]==id]["target_seq"].item()

NameError: name 'i' is not defined

In [11]:
whole_list = []
for t in range(3):
    filename = f"substitution_part{t+1}.json"
#os.file.exist(json_path+filename)
    with open(json_path+filename, 'r') as j:
         contents = json.loads(j.read())
    for i in range(len(contents["results"])):
        id = contents["results"][i]['xref'][0]['name']
        sequence = contents["results"][i]["sequence"]
        assert sequence == df[df["DMS_id"]==id]["target_seq"].item()
        labels = np.zeros(len(sequence))
        #print(labels)
        for j in range(len(contents["results"][i]["matches"])):
            temp = contents["results"][i]['matches'][j]["signature"]
            location = contents["results"][i]['matches'][j]["locations"]

            #entry_type = temp['entry']['type']
                #if entry_type=="DOMAIN":
                #print(contents["results"][i]['matches'][j]["signature"]['entry'])
            if location[0]['representative'] is True:
                if len(location[0]["location-fragments"])==1 and location[0]["location-fragments"][0]["dc-status"]=='CONTINUOUS':
                    start = int(location[0]["location-fragments"][0]["start"])
                    end = int(location[0]["location-fragments"][0]["end"])
                    if np.all(labels[start-1:end]==0): #should have no overlap between domain
                        labels[start-1:end]=1
                    else:
                        print(f"have overlap:{location}")
                else:
                    print(location[0]["location-fragments"])
        if np.all(labels==0):
            #labels = np.NaN
            print(f"this protein has no domain:{id,i,t,len(sequence)}")
        whole_list.append([id,labels])
print(len(whole_list))
df_domain = pd.DataFrame(whole_list,columns=["DMS_id","representative_domain_mask"])

    

this protein has no domain:('HCP_LAMBD_Tsuboyama_2023_2L6Q', 5, 0, 55)
this protein has no domain:('A0A2Z5U3Z0_9INFA_Doud_2016', 11, 0, 565)
this protein has no domain:('GLPA_HUMAN_Elazar_2016', 23, 0, 150)
this protein has no domain:('BCHB_CHLTE_Tsuboyama_2023_2KRU', 38, 0, 52)
this protein has no domain:('A0A247D711_LISMN_Stadelmann_2021', 40, 0, 87)
this protein has no domain:('GDIA_HUMAN_Silverstein_2021', 45, 0, 447)
this protein has no domain:('C6KNH7_9INFA_Lee_2018', 51, 0, 566)
this protein has no domain:('MAFG_MOUSE_Tsuboyama_2023_1K1V', 2, 1, 41)
this protein has no domain:('KCNE1_HUMAN_Muhammad_2023_function', 10, 1, 129)
this protein has no domain:('MLAC_ECOLI_MacRae_2023', 13, 1, 211)
this protein has no domain:('LGK_LIPST_Klesmith_2015', 25, 1, 439)
this protein has no domain:('NUSA_ECOLI_Tsuboyama_2023_1WCL', 39, 1, 69)
this protein has no domain:('PA_I34A1_Wu_2015', 45, 1, 716)
this protein has no domain:('NCAP_I34A1_Doud_2015', 54, 1, 498)
this protein has no domain:('

In [30]:
from scipy.stats import spearmanr
import numpy as np
dms_id_ignored = ["A0A140D2T1_ZIKV_Sourisseau_2019", "BRCA2_HUMAN_Erwood_2022_HEK293T", "CAS9_STRP1_Spencer_2017_positive", "P53_HUMAN_Giacomelli_2018_Null_Etoposide", "P53_HUMAN_Giacomelli_2018_Null_Nutlin", "P53_HUMAN_Giacomelli_2018_WT_Nutlin",
"POLG_HCVJF_Qi_2014"]
ProtSSN_result = "/scratch/user/yxliu/ecen766/course_project/benchmark/ProtSSN/result/ProteinGym/"

domain_spearman_list = []
other_spearman_list = []
for i in range(len(df)):
    dms_id = df.iloc[i]['DMS_id']
    includes_multiple_mutants = df.iloc[i]['includes_multiple_mutants']
    if (dms_id not in dms_id_ignored) and ~includes_multiple_mutants:
        domain_gt = []
        domain_pred = []
        other_gt = []
        other_pred = []
        df_protssn = pd.read_csv(ProtSSN_result + dms_id + ".csv")
        gt = df_protssn["score"].tolist()
        mutation_sites = df_protssn["mutant"].tolist()
        mutation_sites = [int(x[1:-1]) for x in mutation_sites]
        pred_protssn = df_protssn["ProtSSN_k20_h512"].tolist()
        if dms_id in df_domain["DMS_id"].tolist():
            domain_mask = df_domain[df_domain["DMS_id"]==dms_id]["representative_domain_mask"].item()
        else:
            seq = df[df["DMS_id"]==dms_id]["target_seq"].item()
            same_seq_dms_ids = df[df["target_seq"]==seq]["DMS_id"].tolist()
            for same_seq_dms_id in same_seq_dms_ids:
                if same_seq_dms_id in df_domain["DMS_id"].tolist():
                    domain_mask = df_domain[df_domain["DMS_id"]==same_seq_dms_id]["representative_domain_mask"].item()
        #print(dms_id)
        domain_residues = np.where(domain_mask==1)[0]
        other_residues = np.where(domain_mask!=1)[0]
        #print(len(domain_residues),len(other_residues),len(df.iloc[i]['target_seq']))
        for i in range(len(mutation_sites)):
            if mutation_sites[i] in domain_residues:
                domain_gt.append(gt[i])
                domain_pred.append(pred_protssn[i])
            if mutation_sites[i] in other_residues:
                if dms_id!="CCDB_ECOLI_Adkar_2012":
                    other_gt.append(gt[i])
                    other_pred.append(pred_protssn[i])
        if len(domain_gt) > 1:
            domain_spearman_list.append(spearmanr(domain_gt, domain_pred)[0])
        if len(other_gt) > 1:
            #print(dms_id,other_gt,other_pred)
            other_spearman_list.append(spearmanr(other_gt, other_pred)[0])

print( np.mean(domain_spearman_list), np.mean(other_spearman_list),len(domain_spearman_list),len(other_spearman_list))

0.45952904165689673 0.26762370493673854 110 112


In [ ]:
filename = f"substitution_part{t+1}.json"
#os.file.exist(json_path+filename)
with open(json_path+filename, 'r') as j:
    contents = json.loads(j.read())
for i in range(1):
    i=46
    id = contents["results"][i]['xref'][0]['name']
    sequence = contents["results"][i]["sequence"]
    print(id,sequence)
    assert sequence == df[df["DMS_id"]==id]["target_seq"].item()
    labels = np.zeros(len(sequence))
        #print(labels)
    for j in range(len(contents["results"][i]["matches"])):
        temp = contents["results"][i]['matches'][j]["signature"]
        location = contents["results"][i]['matches'][j]["locations"]
        print(len(temp))
        print(location[0]["start"],location[0]["end"])
        if temp['entry'] is not None:
            entry_type = temp['entry']['type']
            if entry_type=="DOMAIN":
                print(contents["results"][i]['matches'][j]["signature"]['entry'])
                print(location[0]["start"],location[0]["end"])

In [9]:
t=2
i=46
filename = f"substitution_part{t+1}.json"
#os.file.exist(json_path+filename)
with open(json_path+filename, 'r') as j:
    contents = json.loads(j.read())
id = contents["results"][i]['xref'][0]['name']
print(id)
sequence = contents["results"][i]["sequence"]
print(sequence)
assert sequence == df[df["DMS_id"]==id]["target_seq"].item()
labels = np.zeros(len(sequence))
        #print(labels)
for j in range(len(contents["results"][i]["matches"])):
    print(j)
    temp = contents["results"][i]['matches'][j]["signature"]
    location = contents["results"][i]['matches'][j]["locations"]
    print(location)
    if temp['entry'] is not None:
        entry_type = temp['entry']['type']
        #print(entry_type)
        #print(location[0])

SHOC2_HUMAN_Kwon_2022
MSSSLGKEKDSKEKDPKVPSAKEREKEAKASGGFGKESKEKEPKTKGKDAKDGKKDSSAAQPGVAFSVDNTIKRPNPAPGTRKKSSNAEVIKELNKCREENSMRLDLSKRSIHILPSSIKELTQLTELYLYSNKLQSLPAEVGCLVNLMTLALSENSLTSLPDSLDNLKKLRMLDLRHNKLREIPSVVYRLDSLTTLYLRFNRITTVEKDIKNLSKLSMLSIRENKIKQLPAEIGELCNLITLDVAHNQLEHLPKEIGNCTQITNLDLQHNELLDLPDTIGNLSSLSRLGLRYNRLSAIPRSLAKCSALEELNLENNNISTLPESLLSSLVKLNSLTLARNCFQLYPVGGPSQFSTIYSLNMEHNRINKIPFGIFSRAKVLSKLNMKDNQLTSLPLDFGTWTSMVELNLATNQLTKIPEDVSGLVSLEVLILSNNLLKKLPHGLGNLRKLRELDLEENKLESLPNEIAYLKDLQKLVLTNNQLTTLPRGIGHLTNLTHLGLGENLLTHLPEEIGTLENLEELYLNDNPNLHSLPFELALCSKLSIMSIENCPLSHLPPQIVAGGPSFIIQFLKMQGPYRAMV
0
[{'start': 170, 'end': 191, 'representative': False, 'score': 7.942976, 'alignment': 'KLRMLDLRHNKLREIPSVVYRL', 'location-fragments': [{'start': 170, 'end': 191, 'dc-status': 'CONTINUOUS'}]}]
1
[{'start': 124, 'end': 145, 'representative': False, 'score': 7.265315, 'alignment': 'QLTELYLYSNKLQSLPAEVGCL', 'location-fragments': [{'start': 124, 'end': 145, 'dc-status': 'CONTINUOUS'}]}]
2
[{'star

In [48]:
contents["results"][i]['matches'][j].keys()

dict_keys(['signature', 'locations', 'evalue', 'score', 'model-ac'])

In [10]:
i=9
id = contents["results"][i]['xref'][0]['name']
print(id)
contents["results"][i].keys()
for j in range(len(contents["results"][i]["matches"])):
    temp = contents["results"][i]['matches'][j]["signature"]
    location = contents["results"][i]['matches'][j]["locations"]
    if temp['entry'] is not None:
        entry_type = temp['entry']['type']
        #if entry_type=="DOMAIN":
        print(contents["results"][i]['matches'][j]["signature"]['entry']['type'])
        print(location)

UBE4B_MOUSE_Starita_2013
DOMAIN
[{'start': 1098, 'end': 1171, 'representative': False, 'score': 33.91618, 'alignment': 'DAPDEFRDPLMDTLMTDPVRLPSGTVMDRSIILRHL-LNSPTDPFNRQMLTESMLEPVPELKEQIQAWMREKQSS', 'location-fragments': [{'start': 1098, 'end': 1171, 'dc-status': 'CONTINUOUS'}]}]
DOMAIN
[{'start': 1102, 'end': 1164, 'representative': False, 'hmmStart': 1, 'hmmEnd': 72, 'hmmLength': 72, 'hmmBounds': 'COMPLETE', 'evalue': 5.5e-25, 'score': 99.0, 'location-fragments': [{'start': 1102, 'end': 1164, 'dc-status': 'CONTINUOUS'}]}]
DOMAIN
[{'start': 462, 'end': 1083, 'representative': False, 'hmmStart': 1, 'hmmEnd': 628, 'hmmLength': 628, 'hmmBounds': 'COMPLETE', 'evalue': 0.0, 'score': 720.7, 'envelopeStart': 462, 'envelopeEnd': 1083, 'postProcessed': True, 'location-fragments': [{'start': 462, 'end': 1083, 'dc-status': 'CONTINUOUS'}]}]
HOMOLOGOUS_SUPERFAMILY
[{'start': 1096, 'end': 1173, 'representative': False, 'hmmStart': 3, 'hmmEnd': 76, 'hmmLength': 78, 'hmmBounds': 'COMPLETE', 'evalue': 

In [78]:
print(contents["results"][0]["sequence"])

MSSSSWLLLSLVAVTAAQSTIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSVGLPNMTQGFWENSMLTDPGNVQKAVCHPTAWDLGKGDFRILMCTKVTMDDFLTAHHEMGHIQYDMAYAAQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLSPDFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKDQWMKKWWEMKREIVGVVEPVPHDETYCDPASLFHVSNDYSFIRYYTRTLYQFQFQEALCQAAKHEGPLHKCDISNSTEAGQKLFNMLRLGKSEPWTLALENVVGAKNMNVRPLLNYFEPLFTWLKDQNKNSFVGWSTDWSPYADQSIKVRISLKSALGDKAYEWNDNEMYLFRSSVAYAMRQYFLKVKNQMILFGEEDVRVANLKPRISFNFFVTAPKNVSDIIPRTEVEKAIRMSRSRINDAFRLNDNSLEFLGIQPTLGPPNQPPVSIWLIVFGVVMGVIVVGIVILIFTGIRDRKKKNKARSGENPYASIDISKGENNPGFQNTDDVQTSF
